In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
%matplotlib inline

train = pd.read_csv('sales_train.csv')
test = pd.read_csv('test.csv')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
items_categories = pd.read_csv('item_categories.csv')

In [9]:
scaler = StandardScaler().fit(train['item_price'].values.reshape(-1, 1))
train['item_price'] = scaler.transform(train['item_price'].values.reshape(-1, 1))
scaler = StandardScaler().fit(train['item_cnt_day'].values.reshape(-1, 1))
train['item_cnt_day'] = scaler.transform(train['item_cnt_day'].values.reshape(-1, 1))
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,1.565574e-14,-7.449325e-16
std,9.422988e+00,1.622697e+01,6.324297e+03,1.000000e+00,1.000000e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-5.155819e-01,-8.875187e+00
25%,7.000000e+00,2.200000e+01,4.476000e+03,-3.710565e-01,-9.265226e-02
50%,1.400000e+01,3.100000e+01,9.343000e+03,-2.843412e-01,-9.265226e-02
75%,2.300000e+01,4.700000e+01,1.568400e+04,6.251983e-02,-9.265226e-02
max,3.300000e+01,5.900000e+01,2.216900e+04,1.775288e+02,8.277567e+02
